##Prepare

Get raw data

In [ ]:
!gdown 1D62SLrqV2Qo3amiihH8rDpY_GcZGIL34

Downloading...
From: https://drive.google.com/uc?id=1D62SLrqV2Qo3amiihH8rDpY_GcZGIL34
To: /content/ub_sample_data.csv
100% 1.78M/1.78M [00:00<00:00, 114MB/s]


Install pyspark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 8.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3949f051da5fd212b4d0d9d1be96c6dd75971adda8dd5bc55abc3e000c070939
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Import libraries

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

Initiate pypark

In [ ]:
spark = SparkSession.builder.appName("PysparkExample")\
    .config ("spark.sql.shuffle.partitions", "50")\
    .config("spark.driver.maxResultSize","5g")\
    .config ("spark.sql.execution.arrow.enabled", "true")\
    .getOrCreate()

##Prepare data 

Make spark dataframe from raw data file

In [ ]:
sample_df = spark.read.csv('ub_sample_data.csv', header = True )
sample_df

DataFrame[user_id: string, business_id: string]

In [ ]:
sample_df.show(5)

+--------------------+--------------------+
|             user_id|         business_id|
+--------------------+--------------------+
|39FT2Ui8KUXwmUt6h...|RJSFI7mxGnkIIKiJC...|
|39FT2Ui8KUXwmUt6h...|fThrN4tfupIGetkrz...|
|39FT2Ui8KUXwmUt6h...|mvLdgkwBzqllHWHwS...|
|39FT2Ui8KUXwmUt6h...|uW6UHfONAmm8QttPk...|
|39FT2Ui8KUXwmUt6h...|T70pMoTP008qYLsIv...|
+--------------------+--------------------+
only showing top 5 rows



Register table to use Sql script

In [ ]:
sample_df.registerTempTable("sample_df")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


SQL script to get edge from raw data

In [ ]:
edge_df = spark.sql('''
  select u1, u2
  from 
      (select  df1.user_id u1, df2.user_id u2
      from sample_df df1, sample_df df2
      where df1.business_id = df2.business_id 
        and df1.user_id < df2.user_id)
  group by u1, u2
  having count(u1) >= 7
''')

In [ ]:
edge_df.count()

498

Export edge data to csv to use in task 2.

In [ ]:
# edge_df.toPandas().to_csv('edge_df.csv', index=False)

##Task 1:

####Function used in task 1

In [ ]:
# Somehow, sum() function cannot work
# So I need to loop through the list to get the sum.


def get_data(graph, edge_df):
    # create edges for the graph from edge spark dataframe

    for row in edge_df.collect():
        u1 = row['u1']
        u2 = row['u2']

        if (u1 not in graph.keys()) and (u2 not in graph.keys()):
            # if two users are not in the graph.
            graph[u1] = {u2: 0}
            graph[u2] = {u1: 0}
        elif (u1 not in graph.keys()):
            # If user1 not in the graph
            graph[u1] = {u2: 0}
            graph[u2][u1] = 0
        elif (u2 not in graph.keys()):
            # If user2 not in the graph
            graph[u2] = {u1: 0}
            graph[u1][u2] = 0
        else:
            # If two users are already in the graph
            graph[u1][u2] = 0
            graph[u2][u1] = 0


def reset_weight(graph):
    # reset weight (betweeness) of the graph to 0
    for node in graph.keys():
        neighbors = graph[node].keys()
        weights = [0]*len(neighbors)
        graph[node] = {neighbor: weight for neighbor, weight in zip(neighbors, weights)}


def BFS_gen_tree(graph, root):
    # Use BFS to create a tree
    # Each level is a dictionary
    # Level n + 1 contain child nodes of one or more node in level n

    # current_node = root

    # Setup for level 0
    visited_nodes = [root]
    tree = [{root:{'number_shortest_path': 1, 'credit': 1}}]
    level = 1

    # Loop until cannot add more node into the tree
    while True:
        current_level_nodes = []

        # add nodes to the current level nodes
        for node in tree[level - 1].keys():
            current_level_nodes += [neighbor for neighbor in graph[node] if neighbor not in visited_nodes]

        # add nodes to the tree
        if len(current_level_nodes) > 0:
            visited_nodes += current_level_nodes
        else:
            # If there is no node, stop the loop
            break
        
        # Create current_level
        current_level = {}
        for node in current_level_nodes:
            # Find parents for the node
            parents = [parent for parent in tree[level-1].keys() if parent in graph[node].keys()]

            # get the number of shortest path of the node by loop though its parents
            parent_points = [tree[level -1][parent]['number_shortest_path'] for parent in parents]

            number_shortest_path = 0
            for point in parent_points:
                number_shortest_path += point

            # add the node to current level with its number of shortest path and credit
            # Credit of the node is initiate to 1 because at any point, the credit of each node will be added by 1
            current_level[node] = {'number_shortest_path': number_shortest_path, 'credit': 1}

        # append current level to the tree
        tree.append(current_level)

        # increase level
        level += 1
    return tree


def calc_betweeness(graph, tree):
    # calculate the betweeness of each edge in the graph with the BFS tree

    # number of level is n
    # level number run from n-1 to 1
    # at each level calculate the betweeness of the edges that lead to the parent's level
    # and calculate the credit of parent nodes
    for level_num in range(len(tree) - 1, 0, -1):
        # get the current level
        level = tree[level_num]

        # get the parent level
        parent_level = tree[level_num -1]

        # loop through the current level and compute for the parent level
        for node in level:
            # get the parents of the node
            parents = [parent for parent in parent_level if parent in graph[node].keys()]

            # calculate credit for the edges lead to the parents
            credit = level[node]['credit'] / len(parents)

            # compute credit and betweeness for the parents and edges
            for parent in parents:
                # add credit for the parent nodes
                parent_level[parent]['credit'] += credit

                # add betweeness for the edges
                # devide by 2 because each edge will be compute 2 times
                graph[parent][node] += credit/2
                graph[node][parent] += credit/2
    

def remove_between_edge(graph):
    # remove the edge with highest betweenness

    # setup highest betweenness and node to remove
    between_edge = (0,0)
    high_betweeness = 0

    # get nodes in the graph.
    nodes = list(graph.keys())

    # loop through each node, get edge's betweenness 
    # and update highest betweenness and node to remove
    for node in nodes:
        # loop through neighbors of the node
        for neighbor in graph[node].keys():
            # get edge's betweenness(edge: [node, neighbor])
            betweeness = graph[node][neighbor]

            # update edge to remove and highest betweenness
            if betweeness > high_betweeness:
                between_edge = (node, neighbor)
                high_betweeness = betweeness

    # remove the edge in the undirected graph 
    # remove two edges: [u1, u2], [u2, u1]
    graph[between_edge[0]].pop(between_edge[1])
    graph[between_edge[1]].pop(between_edge[0])


def get_communities(graph ):
    # Use BFS to get communities
    # add all nodes to the to_visit list.
    # start with a random node in the list to be a root
    # BFS until there is no node to add to the community
    # (nodes are in community is remove from to_visit list)
    # repeat with another node in the to_visit list until there is no node to visit

    # setup
    nodes_to_visit = list(graph.keys())
    communities = []

    # stop when there is no node in to_visit list
    while len(nodes_to_visit) > 0:
        # set a random node to be a root of BFS
        community = [nodes_to_visit[0]]



        # i is an index of node in the community (node_i)
        # each loop will add neighbor of node_i into the community
        # stop when there is no new node in the community(index i out of community range)
        i = 0
        while i < len(community):
            # loop through the neighbor of node_i
            for neighbor in list(graph[community[i]].keys()):
                # add neighbor into community if it is not already in community
                if neighbor not in community:
                    community.append(neighbor)
            # increase index
            i+=1

        # add community to the community list
        communities.append(community)

        # remove node in community from to_visit list
        nodes_to_visit = [node for node in nodes_to_visit if node not in community]

    # return list of communities
    return communities


def modularity(graph, communities, A, m):
    # calculate modularity with the formula in task 1

    # setup
    total_score = 0

    # loop through each community in list of communities
    for community in communities:
        # setup modularity for the cummunity
        score = 0

        # loop through each node in the community
        for node1 in community:
            # get degree of node1
            k1 = len(A[node1])
            
            # loop through each node in the community once more time 
            for node2 in community:
                # get degree of node2
                k2 = len(A[node2])  

                # check if there an edge between two nodes
                Aij = 1 if node2 in A[node1] else 0

                # update modularity of the community
                score += (Aij - (float(k1*k2)/(2*m)))

        # update modularity of graph (communities)
        total_score += score

    # return normalized value of modularity
    return total_score / (2*m)


def modularity2(graph, communities, A, m):
    # calculate modularity follow the shorten formula privide in networkx modularity
    # which was used to calculate the modularity by networkx
    # We do this to try to get the exact number of modularity that networkx produced

    # setup
    total_score = 0

    # loop through each community
    for community in communities:

        # calculate inside link of the community 
        # which is a link that link two nodes inside of the community

        # sum function broken so i loop through the list.
        # inside_degree = sum([sum([1 if neighbor in community else 0 for neighbor in A[node] ]) for node in community ]) / 2

        # calculate inside degree
        inside_degree_list = [[1 if neighbor in community else 0 for neighbor in A[node] ] for node in community ]
        inside_degree = 0
        for row in inside_degree_list:
            for i in row:
                inside_degree += i

        # calculate total inside link which is total inside degree divide by 2
        inside_link = inside_degree / 2

        # calculate total degree of all nodes in community
        
        # full_degree = sum([sum(len(graph[node].keys())) for node in community]) /2
        full_degree_list = [len(A[node]) for node in community]

        full_degree = 0
        for i in full_degree_list:
            full_degree += i

        # calculate modularity of the community 
        # and update modulairty of the comminites
        total_score += ((inside_link / m ) - (full_degree / (2*m ))**2 )

    # return modularity
    return total_score


def girvan_newman(ori_graph , modularity=modularity):
    # copy the graph to compute on just a copy
    # the original graph will not be affect
    graph = ori_graph.copy()

    # define a calculate number of edges in a graph
    # (graph to store data, not a graphic graph)
    def calc_edges(graph, nodes):
        num_edges = 0
        edges_list = [len(list(graph[node].keys())) for node in nodes]
        for edges in edges_list:
            num_edges += edges
        return num_edges

    # set up
    nodes = list(graph.keys())

    # neighbors_list is a list of neighber respected to node in nodes
    neighbors_list = [list(graph[node].keys()) for node in nodes]

    # get the original adjacent list and the number of edges of the graph
    A = {node: neighbors for node, neighbors in zip(nodes, neighbors_list)}
    m = calc_edges(graph, nodes)/2
    
    # set up
    best_modularity = -1000
    best_communities = []


    # loop until there is no edge in the graph
    while calc_edges(graph, nodes) > 0:
        # loop through the nodes
        for node in nodes:
            # generate tree with root is the current node
            tree = BFS_gen_tree(graph, node)

            # calculate betweenness with the tree just maked
            calc_betweeness(graph, tree)

        # remove the edge with highest betweenness
        remove_between_edge(graph)

        # reset edge's betweenness to 0
        reset_weight(graph)

        # get the communities after remove the edge
        communities = get_communities(graph)

        # calculate modularity of the graph
        score = modularity(graph, communities, A, m)

        # update the best modularity
        # and the communities with the best modularity
        if score > best_modularity:
            best_modularity = score
            best_communities = communities

    # return the best communities in the formated follow the instruction and its modularities
    return (sorted(sorted([sorted(community) for community in best_communities],key=lambda x: x[0] ), key=len), best_modularity)

#### Create a graph and get data for it

In [ ]:
graph = {}
get_data(graph, edge_df)

#### Task 1.1

In [ ]:
# get nodes in graph
nodes = list(graph.keys())

# loop through each node
for node in nodes: 
    # make BFS tree with the current node is root
    tree = BFS_gen_tree(graph, node)

    # calculate betweennes of the edges
    calc_betweeness(graph, tree)

In [ ]:
# sort nodes 
nodes.sort()

# visited nodes store all the nodes that have visited 
# because there is 2 edge represent for 1 real each: 
# Ex: [u1,u2] is the same as [u2,u1]
# when we get all the edges like: [u1, *]
# add u1 to the visited nodes so that
# those edges like: [*, u1], will not be get.

visited_nodes = []
result1 = []

# loop through the nodes
for node in nodes:
    # if node is not visit
    if node not in visited_nodes:
        # get all the edges (like [node, neighbor]) with its betweenness 
        # if the neighbor is not in visited_nodes
        temp_result1 = [[(node, neighbor), graph[node][neighbor]] for neighbor in list(graph[node].keys()) if neighbor not in visited_nodes]

        # add those to the result
        result1 += temp_result1

        # add node to visited
        visited_nodes.append(node)

In [ ]:
# sort descending by betweennes
result1.sort(key=lambda x: x[1], reverse=True)

# write result to file
result1 = [', '.join([str(i) for i in item]) for item in result1]
with open("task1_1_result.txt", "w") as file1:
    file1.write('\n'.join(result1))

In [ ]:
# reset betweeennes for next task
reset_weight(graph)

#### Task 1.2

In [ ]:
# run girvan_newman and print out best_modularity
best_communities, best_modularity = girvan_newman(graph)
best_modularity

0.6872550442734674

In [ ]:
# write result to file
result = str(best_communities).replace('], ','\n').replace('[','').replace(']]','')
with open("task1_2_result.txt", "w") as file2:
    file2.write(result)
    file2.write('\n'+str(best_modularity))

##### Try to replicate the modulairty of networkx module

In [ ]:
# graph2 = graph.copy()
graph2 = {}
get_data(graph2, edge_df)
best_communities, best_modularity = girvan_newman(graph2, modularity=modularity2)
best_modularity

0.6872550442734798

In [ ]:
# write result to file
result = str(best_communities).replace('], ','\n').replace('[','').replace(']]','')
with open("task1_2_result2.txt", "w") as file2:
    file2.write(result)
    file2.write('\n'+str(best_modularity))